In [30]:
import os
import json
import time
import requests
import csv, io
import boto3
import pytz
# import urllib.request
import pandas as pd
from datetime import datetime, timedelta

CLIENT_ID = "RanDo_fK1wsndxplSaxq"
CLIENT_SECRET = "6VZo9TmH9Z"

In [35]:
from urllib.parse import quote
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as bs

def get_results_xml(start):
    start = str(start)
    #start = 검색 시작 위치 (default = 1)
    encText = urllib.parse.quote("LIG넥스원 수출")
    display = '100' # 100이 최대
    sort = 'date' # 날짜순으로 내림차순 정렬
    url = ("https://openapi.naver.com/v1/search/news.xml?" +
       "query=" + encText +
       "&display=" + display +
        "&start=" + start +
       "&sort=" + sort)
    request = Request(url)
    request.add_header("X-Naver-Client-Id",CLIENT_ID)
    request.add_header("X-Naver-Client-Secret",CLIENT_SECRET)
    response = urlopen(request).read()
    soup = bs(response, "xml")
    news=[]
    for item in soup.find_all('item') :
        news.append({"제목": item.find("title").text, 
                     '발행일자': item.find('pubDate').text,
                     "요약": item.find("description").text,
                     'Link':item.find("originallink").text})

    df = pd.DataFrame(news)
    df['발행일자'] = pd.to_datetime(df['발행일자'], format='%a, %d %b %Y %H:%M:%S %z')
    df['발행일자'] = df['발행일자'].dt.strftime("%Y-%m-%d")
    return df

result_df = pd.DataFrame()
for i in range(11):
    if i == 10:
        start = 1000
        df = get_results(start)
        result_df = pd.concat([result_df, df])
    else:
        start = 1 + i*100   #1, 101, 201, ... 901
        df = get_results(start)
        result_df = pd.concat([result_df, df])

result_df.to_csv('output_xml.csv', index=False)

In [33]:
news_df

,제목,발행일자,요약,Link
0,"[산업 이모저모] SK이노 환경과학기술원, 전과정평가로 제품 생애주기 전 단계...","Fri, 21 Jul 2023 14:31:00 +0900",선정된 기업은 연구개발(R&amp;D) 영역에서 국비 과제 기반 지원(최대 4년)을...,http://www.greened.kr/news/articleView.html?id...
1,"한국산 무기, 왜 폴란드에 대거 <b>수출</b>될까","Fri, 21 Jul 2023 12:36:00 +0900",방산 <b>수출</b>계약도 논의 중이다. 폴란드가 K-방산 무기를 대거 수입하는 ...,https://daily.hankooki.com/news/articleView.ht...
2,"<b>LIG넥스원</b>의 무인수상정, 글로벌 시장 진출 노린다","Fri, 21 Jul 2023 11:13:00 +0900",또한 K-방산의 <b>수출</b> 흐름과 함께 새로운 <b>수출</b> 품목을 얻게...,https://economist.co.kr/article/view/ecn202307...
3,"<b>LIG넥스원</b>·풍산, 네덜란드 &apos;큰 손&apos; 블랙리스트…&...","Fri, 21 Jul 2023 10:52:00 +0900",뉴데일리 이가영 기자 = <b>LIG넥스원</b>과 풍산이 글로벌 투자 큰손의 블랙...,https://biz.newdaily.co.kr/site/data/html/2023...
4,"[IPO 프리즘] 코츠테크, 공모 임박…FI 웃음꽃 필까","Fri, 21 Jul 2023 06:30:00 +0900","주요 고객사는 <b>LIG넥스원</b>과 한화시스템, 현대로템 등 국내 방산업체다....",https://dealsite.co.kr/articles/107201
...,...,...,...,...
95,과학도시 대전에서 첨단 국방부품·장비 전시 개막,"Wed, 28 Jun 2023 15:52:00 +0900","한화·<b>LIG</b> 등 무인헬기, 무인전투차, 드론 통합체계 대거 선봬 방산 ...",https://www.munhwa.com/news/view.html?no=20230...
96,"<b>LIG넥스원</b>, 국방산업전 참가...헬기용 유도로켓 소개","Wed, 28 Jun 2023 15:12:00 +0900",방산<b>수출</b> 선순환 구조 마련을 위해 대전시가 매년 개최해 온 &apos;...,http://www.finomy.com/news/articleView.html?id...
97,대전서 첨단국방산업전 막 올랐다,"Wed, 28 Jun 2023 15:00:00 +0900","한화, <b>LIG넥스원</b>, 풍산 등 체계기업(방산 대기업)에서는 공병전투차량...",http://www.goodmorningcc.com/news/articleView....
98,대전서 &apos;2023 첨단 국방부품·장비 전시&apos; 개막,"Wed, 28 Jun 2023 14:58:00 +0900","한화, <b>LIG넥스원</b>, 풍산 등 체계기업(방산 대기업)에서는 공병전투차량...",http://www.jbnews.com/news/articleView.html?id...


In [25]:
# 뉴스 검색

def get_results(start):
    start = str(start)
    #start = 검색 시작 위치 (default = 1)
    encText = urllib.parse.quote("LIG넥스원 수출")
    display = '100' # 100이 최대
    sort = 'date' # 날짜순으로 내림차순 정렬
    url = ("https://openapi.naver.com/v1/search/news.json?" +
       "query=" + encText +
       "&display=" + display +
        "&start=" + start +
       "&sort=" + sort)
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",CLIENT_ID)
    request.add_header("X-Naver-Client-Secret",CLIENT_SECRET)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        json_data = response_body.decode('utf-8')
        data_dict = json.loads(json_data)
        df = pd.DataFrame(data_dict['items'])
    else:
        print("Error Code:" + rescode)
    df['pubDate'] = pd.to_datetime(df['pubDate'], format='%a, %d %b %Y %H:%M:%S %z')
    df['pubDate'] = df['pubDate'].dt.strftime("%Y-%m-%d")
    return df

result_df = pd.DataFrame()
for i in range(11):
    if i == 10:
        start = 1000
        df = get_results(start)
        result_df = pd.concat([result_df, df])
    else:
        start = 1 + i*100   #1, 101, 201, ... 901
        df = get_results(start)
        result_df = pd.concat([result_df, df])

result_df.to_csv('output.csv', index=False)


In [26]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         1100 non-null   object
 1   originallink  1100 non-null   object
 2   link          1100 non-null   object
 3   description   1100 non-null   object
 4   pubDate       1100 non-null   object
dtypes: object(5)
memory usage: 51.6+ KB


In [28]:
import requests

url = ('https://newsapi.org/v2/everything?'+
       'q=apple&'+
       'to=2023-07-22&'+
       'sortBy=publishedAt&'+
       'apiKey=f245f5cacef4446c820f407bca55477c')

response = requests.get(url)

if response.status_code == 200:
    data_json = response.json()
    df_news = pd.DataFrame(data_json['articles'])
    print(df_news)
else:
    print("Error: Unable to fetch data from the API.")


                                               source  \
0   {'id': None, 'name': 'International Business T...   
1                     {'id': None, 'name': 'TheGrio'}   
2               {'id': None, 'name': 'Tecnoblog.net'}   
3                     {'id': None, 'name': '9to5Mac'}   
4        {'id': 'google-news', 'name': 'Google News'}   
..                                                ...   
95     {'id': 'ars-technica', 'name': 'Ars Technica'}   
96            {'id': None, 'name': 'Journal du geek'}   
97                     {'id': None, 'name': 'Forbes'}   
98                   {'id': None, 'name': 'Feber.se'}   
99                   {'id': None, 'name': '9to5Toys'}   

                                               author  \
0                                            AFP News   
1                               Linda Earley Chastang   
2                                     Emerson Alecrim   
3                                         Ben Lovejoy   
4                             

In [29]:
df_news.head()

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': None, 'name': 'International Business T...",AFP News,"After Hollywood Strike, Broadway Staves Off In...",A tentative agreement was reached in New York ...,https://www.ibtimes.com/after-hollywood-strike...,https://d.ibtimes.com/en/full/4474912/averted-...,2023-07-21T12:28:13Z,An averted Broadway strike would have closed 2...
1,"{'id': None, 'name': 'TheGrio'}",Linda Earley Chastang,What it means to have John Lewis on a U.S. stamp,OPINION: Having the face of the civil rights i...,http://thegrio.com/2023/07/21/what-it-means-to...,https://thegrio.com/wp-content/uploads/2021/01...,2023-07-21T12:26:29Z,OPINION: Having the face of the civil rights i...
2,"{'id': None, 'name': 'Tecnoblog.net'}",Emerson Alecrim,Como saber quantos núcleos tem o meu processador?,Número de núcleos da CPU pode impactar desempe...,https://tecnoblog.net/responde/como-ver-quanto...,https://files.tecnoblog.net/wp-content/uploads...,2023-07-21T12:24:52Z,O número de núcleos do processador pode ser en...
3,"{'id': None, 'name': '9to5Mac'}",Ben Lovejoy,Limited iPhone 15 availability: Which would be...,Two reports yesterday suggested that there mig...,https://9to5mac.com/2023/07/21/limited-iphone-...,https://i0.wp.com/9to5mac.com/wp-content/uploa...,2023-07-21T12:23:27Z,Two reports yesterday suggested that there mig...
4,"{'id': 'google-news', 'name': 'Google News'}",None,Limited iPhone 15 availability: Which would be...,<ol><li>Limited iPhone 15 availability: Which ...,https://consent.google.com/ml?continue=https:/...,None,2023-07-21T12:23:00Z,We use cookies and data to<ul><li>Deliver and ...
